In [ ]:
url = "http://127.0.0.1:5000/home"

In [ ]:
import requests
import re
import json
from typing import Dict

def extract_server_name(response):
    message = response.get("message", "")
    match = re.search(r'Server: (\w+)', message)

    if match:
        return match.group(1)
    else:
        return None


def make_requests(count: int) -> Dict[str, int]:

    """
    In this task, you test and analyze the performance of your load balancer implementation in different scenarios. 
    """
    server_count = {}
    for i in range(count):
        try:
            response = requests.get(url)

            server_name = extract_server_name(response.json())

            if server_name:

                if server_name in server_count.keys():
                    server_count[server_name] += 1
                else:
                    server_count[server_name] = 1
        except:
            pass
    return server_count


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
sns.set_theme(style="whitegrid", color_codes=True)


In [ ]:
def draw_bar_chart(server_count: Dict[str, int]):

    """
    Launch 10000 async requests on N = 3 server containers and report the request count handled by each server instance 
    in a bar chart. Explain your observations in the graph and your view on the performance.
    """

    plt.title("Experiment 1: Launch 10000 async requests on N = 3")
    plt.xlabel("Server Name")
    plt.ylabel("Request Count")
    
    sns.barplot(x=server_count.keys(), y=server_count.values(),palette=['blue', 'blue', 'blue', 'purple','red','purple'])
    plt.show()
    

In [ ]:
requests =   make_requests(10000)


In [ ]:
requests

In [ ]:
draw_bar_chart(requests)

In [ ]:
def draw_bar_chart(server_count: Dict[str, int]):

    """
    Launch 10000 async requests on N = 3 server containers and report the request count handled by each server instance 
    in a bar chart. Explain your observations in the graph and your view on the performance.
    """

    plt.title("Experiment 1: Launch 10000 async requests on N = 3")
    plt.xlabel("Server Name")
    plt.ylabel("Request Count")
    
    sns.barplot(x=server_count.keys(), y=server_count.values(),palette=['blue', 'purple', 'blue', 'blue','purple','blue'])
    plt.show()
    

In [ ]:
six_servers_request = make_requests(10000)

In [ ]:
draw_bar_chart(six_servers_request)

In [ ]:
six_servers_request2 = make_requests(20000)

In [ ]:
def draw_line_chart(two,three,four,five,six):
    """
    Next, increment N from 2 to 6 and launch 10000 requests on each such increment. Report the average load of the servers 
    at each run in a line chart.
    """
    plt.plot(two.keys(), two.values(), label="2 Servers")
    plt.plot(three.keys(), three.values(), label="3 Servers")
    plt.plot(four.keys(), four.values(), label="4 Servers")
    plt.plot(five.keys(), five.values(), label="5 Servers")
    plt.plot(six.keys(), six.values(), label="6 Servers")
    plt.title("Experiment 2: Scalability")
    plt.xlabel("Server")
    plt.ylabel("Request Count")
    plt.legend()
    plt.show()

In [ ]:
draw_bar_chart(six_servers_request2)

In [ ]:
import requests
import matplotlib.pyplot as plt
from collections import defaultdict

# Function to simulate sending requests to the load balancer
def send_requests(load_balancer_url, num_requests=10000):
    server_load = defaultdict(int)
    for _ in range(num_requests):
        response = requests.get(f'{load_balancer_url}/home')
        if response.status_code == 200:
            server = extract_server_name(response.json())
            server_load[server] += 1
    return server_load

# Function to plot the results
def plot_results(results):
    x = list(results.keys())
    y = [sum(loads.values()) / len(loads) for loads in results.values()]
    
    plt.figure(figsize=(10, 6))
    plt.plot(x, y, marker='o')
    plt.xlabel('Number of Servers')
    plt.ylabel('Average Load (requests per server)')
    plt.title('Scalability Experiment: Average Load Distribution')
    plt.grid(True)
    plt.show()

# Main experiment loop
def scalability_experiment(load_balancer_url, initial_servers=2, max_servers=6, num_requests=10000):
    results = {}
    
    for num_servers in range(initial_servers, max_servers + 1):
        # Add servers to the load balancer
        hostnames = [f'S{i}' for i in range(num_servers)]
        data = {'n': len(hostnames), 'hostnames': hostnames}
        print(data)
        res = requests.post(f'{load_balancer_url}/add', json=data)
        if res.status_code >=210:
            raise Exception(f"Failed to add servers: {res.text}")
        
        # Send requests and record load
        server_load = send_requests(load_balancer_url, num_requests)
        results[num_servers] = server_load
        
        # Print the load distribution for each run
        print(f'Number of Servers: {num_servers}')
        for server, load in server_load.items():
            print(f'{server}: {load} requests')
        print('-' * 40)

        requests.delete(f'{load_balancer_url}/rm',json=data)
        
    # Plot the results
    plot_results(results)

# URL of the load balancer

# Run the experiment
scalability_experiment("http://localhost:5000")
